<a href="https://colab.research.google.com/github/trismegisto78/romeo/blob/master/testBidirLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import numpy as np
from random import random
from numpy import array
from numpy import cumsum
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import Flatten,CuDNNLSTM
from keras.initializers import glorot_uniform


In [46]:
!git clone https://github.com/trismegisto78/romeo

fatal: destination path 'romeo' already exists and is not an empty directory.


MOUNT GOOGLE DRIVE

In [47]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
with open('/content/gdrive/My Drive/strMacd2004.csv', 'r') as f:
  lerighe = f.read().split('\n')
del lerighe[0]

In [60]:
righe_file=[]
for k in range(0,1000000) :
  righe_file.append(float(lerighe[k].split(";")[1]) )
  
print(righe_file[0])



1.2596


In [0]:
# create a sequence classification instance
def get_sequence(dataset_array):
  idx = np.random.randint(0,len(dataset_array)-130  )
  X,y = get_sequence_from_idx(dataset_array,idx)
  return X,y

def get_sequence_from_idx(dataset_array,idx):
  
  X = np.zeros(n_timesteps)
  Y = np.zeros(1)
   
  
  for k in range(idx,idx+n_timesteps ):
    value=normValue(dataset_array[k])
    X[k-idx]=value
    
  finalValue=normValue(dataset_array[idx+130])

  if finalValue<(value-0.00050):
    Y[0]=1.0
  else:
    Y[0]=0.0
  
  X = X.reshape(n_timesteps,1)
  Y = Y.reshape(1)
  return X, Y

def get_a_batch(batchSize,data_set):
  totexamples = batchSize
  xInput = np.zeros( (totexamples,n_timesteps,1) )
  target = np.zeros( (totexamples,1) )
  for k in range(0,totexamples):
    X,y = get_sequence(data_set)
    xInput[k] = X
    target[k] = y
  
  return xInput,target

def get_simple_random_batch(batchSize,data_set):
  idx = np.random.randint(0,len(data_set),batchSize  )
  fullBatch = data_set[idx]
  
  xInput = np.zeros( (batchSize,n_timesteps,1) )
  target = np.zeros( (batchSize,1) )
  
  for k in range( len(fullBatch)  ):
    xInput[k] = fullBatch[k][0]
    target[k] = fullBatch[k][1]
  
  return xInput,target

def get_simple_all_dataset(data_set):
  
  xInput = np.zeros( (len(data_set),n_timesteps,1) )
  target = np.zeros( (len(data_set),1) )
  
  for k in range( len(data_set)  ):
    xInput[k] = data_set[k][0]
    target[k] = data_set[k][1]
  
  return xInput,target

def getAllDataSet(data_set):
  totexamples = len(data_set)-130
  xInput = np.zeros( (totexamples,n_timesteps,1) )
  target = np.zeros( (totexamples,1) )
  listone = []
  for k in range(0,totexamples):
    X,y = get_sequence_from_idx(data_set,k)
#    xInput[k] = X
#    target[k] = y
    listone.append([X,y])
    if k % 100000 ==0 :
      print('generate %d dataset items ' % k)
  return listone#xInput,target 


In [83]:
from sklearn.model_selection import train_test_split


allDataset = getAllDataSet(righe_file)


train, validation = train_test_split(allDataset,shuffle=False)

print("TRAIN LEN ",len(train) )
print("validation LEN ",len(validation) )


generate 0 dataset items 
generate 100000 dataset items 
generate 200000 dataset items 
generate 300000 dataset items 
generate 400000 dataset items 
generate 500000 dataset items 
generate 600000 dataset items 
generate 700000 dataset items 
generate 800000 dataset items 
generate 900000 dataset items 
TRAIN LEN  749902
validation LEN  249968


In [97]:
#train[0].head()
array_train = np.asarray(train )
array_val = np.asarray(validation )



(100, 1)

In [103]:
#piccolo test
idx = np.random.randint(0,len(array_train),5  )
fullBatch = array_train[idx]
fullBatch[1][1]

array([1.])

In [106]:
n_timesteps = 100

model = Sequential()
model.add(Bidirectional(CuDNNLSTM(n_timesteps, return_sequences=True,kernel_initializer=glorot_uniform()), 
                        input_shape=(n_timesteps, 1), name='inputBIDI'))
model.add(TimeDistributed(Dense(1, activation='tanh')))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid')  ) 

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputBIDI (Bidirectional)    (None, 100, 200)          82400     
_________________________________________________________________
time_distributed_6 (TimeDist (None, 100, 1)            201       
_________________________________________________________________
flatten_6 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 101       
Total params: 82,702
Trainable params: 82,702
Non-trainable params: 0
_________________________________________________________________


In [64]:
ilMax = max(train)
ilMin = min(train)


delta = ilMax-ilMin


print('MAX ', ilMax )
print('MIN ', ilMin )
print('delta ', delta )

MAX  1.3663
MIN  1.164
delta  0.20230000000000015


In [65]:
def normValue(theValue):
  return (float(theValue)-ilMin)/delta

idx = np.random.randint(0, len(train)-130, n_timesteps  )
print(idx)

[245478 250128 476213 264082 313437 263287 183998 733622 494422 336396
 166249 344427 468639 407798 334653 739978 621269 209821 178227 293459
 626150 324512 717362 451720 340951 279838 483148 385814  92884 317170
  66446 265904 192342 469739 456214 285481 130387 634933 631163  68689
 627871 403587 263726 250903 262187 347968 211851 490574 348890 346244
 370379 535469  40081 691278 651940 222156 648267 557874 476067 376979
 650307 532802 403355 433862 362162 316796 721318 172738 380237 424690
 156408 542718 634119 273725   2299 703958 318287 630147 539016 697969
   9007  91413 474083 420573 682892 441516 471670 218792 300204 253147
 222786 348622 749236 197077  60979 664489 702726 179129 542483 497585]


In [117]:
samplePos=0
# train LSTM

#xInput,target = getAllDataSet(validation)

for epoch in range(1000):
 
  X,y = get_simple_random_batch(75,array_train )
  resTrain = model.train_on_batch(X,y)

  if(epoch % 100 == 0):
    print('Apprendimento epoch->',epoch,' ',resTrain)
    
  if(epoch % 1000 == 0 ):
    totexamples = 1000

    xInput,target = get_simple_all_dataset(validation)

    resulto = model.evaluate(xInput, target, verbose=1,batch_size=512)

    print('Eseguo validazione %.2f%%'%(resulto[1]*100), '>>>>>',resulto  )
    print('')

  
# evaluate LSTM



Apprendimento epoch-> 0   [0.6662728, 0.62666667]
249968/249968 [==============================] - 13s 53us/step
Eseguo validazione 60.31% >>>>> [0.6730683977538444, 0.6031131985008522]

Apprendimento epoch-> 100   [0.68626535, 0.56]
Apprendimento epoch-> 200   [0.73812497, 0.46666667]
Apprendimento epoch-> 300   [0.7029981, 0.50666666]


KeyboardInterrupt: ignored